In [1]:
import numpy as np
import argparse
import time
import cv2
import os

In [2]:
labelsPath = "C:\\Users\\deepi\\Desktop\\Research\\Accident dataset\\yolo-object-detection\\yolo-coco\\coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

In [3]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")
weightsPath = "C:\\Users\\deepi\\Desktop\\Research\\Accident dataset\\yolo-object-detection\\yolo-coco\\yolov3.weights"
configPath = "C:\\Users\\deepi\\Desktop\\Research\\Accident dataset\\yolo-object-detection\\yolo-coco\\yolov3.cfg"
outputPath = "C:\\Users\\deepi\\Desktop\\Research\\Accident dataset\\yolo-object-detection\\output"

In [4]:
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

[INFO] loading YOLO from disk...


In [5]:
vs = cv2.VideoCapture("C:\\Users\\deepi\\Desktop\\Research\\Accident dataset\\yolo-object-detection\\videos\\test.mp4")
writer = None
(W, H) = (None, None)

In [13]:
from deepsort import *
deepsort = deepsort_rbc()

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [19]:
deepsort = deepsort_rbc()


while True:
    (grabbed, frame) = vs.read()

    if not grabbed:
        print("Frames not grabbed")
        break

    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()

    boxes = []
    confidences = []
    classIDs = []

    for output in layerOutputs:
        for detection in output:

            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if confidence > 0.3:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.3, 0.5)
    if len(idxs) > 0:
        
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]],confidences[i])
            cv2.putText(frame, text, (x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
    cv2.imshow('frame',frame)
    cv2.waitKey(10000)
    cv2.destroyAllWindows()
print("[INFO] cleaning up...")
# writer.release()
vs.release()

KeyboardInterrupt: 